This notebook is used to combine the processed results file per job into one file, which is then quicker to load than many separate files.

In [ ]:
#TODO: absorb this into one of the main scripts, or load directly from per-job pickles.

%load_ext autoreload 
%autoreload 2

from pathlib import Path,PurePath
from hydra.utils import to_absolute_path
import numpy as np
import torch
from sbi_ice.loaders import BaseLoader,ShortProfileLoader
from sbi_ice.utils.modelling_utils import regrid
from sbi_ice.utils.noise_model import best_contour,depth_error
import pickle
import matplotlib.pyplot as plt


In [ ]:
shelf = "Ekstrom"
exp = "exp2"
# shelf = "Synthetic_long"
# exp = "exp3"
hydra_dir = to_absolute_path("../../out/" + shelf)
setup_dir = to_absolute_path("../../data/" + shelf + "/setup_files")
gt_version = "v0"
selection_method = "advanced_noise"

#Create dataloader for experiment
loader = ShortProfileLoader.ShortProfileLoader(Path(setup_dir),Path(hydra_dir),exp,sims_path ="layer_sims",gt_version=gt_version)

#Iterate through all jobs and combine the processed layers for that job into one file.
contour_arrays_out = []
norm_arrays_out = []
age_arrays_out = []
smb_unperturbed_out = []
smb_cnst_means_out = []
smb_sds_out = []
smb_out = []
bmb_out = []
for job in loader._jobs:
    print(job)
    processed_fname = Path(loader._fol_path, loader._exp_path,"layer_sims",str(job),"all_layers.p")
    mb_fname = Path(loader._fol_path, loader._exp_path,"layer_sims",str(job),"all_mbs.p")

    with open(processed_fname, "rb") as fh:
        contour_arrays,norm_arrays,age_arrays = pickle.load(fh).values()
    with open(mb_fname, "rb") as fh:
        smb_unperturbed_all,smb_cnst_means_all,smb_sds_all,smb_all,bmb_all = pickle.load(fh).values()
    contour_arrays_out.append(contour_arrays)
    norm_arrays_out.append(norm_arrays)
    age_arrays_out.append(age_arrays)
    smb_unperturbed_out.append(smb_unperturbed_all)
    smb_cnst_means_out.append(smb_cnst_means_all)
    smb_sds_out.append(smb_sds_all)
    smb_out.append(smb_all)
    bmb_out.append(bmb_all)

print(contour_arrays_out[0].shape)
print(norm_arrays_out[0].shape)
print(age_arrays_out[0].shape)
print(smb_unperturbed_out[0].shape)
print(smb_cnst_means_out[0].shape)
print(smb_sds_out[0].shape)
print(smb_out[0].shape)
print(bmb_out[0].shape)

contour_arrays_out = torch.cat(contour_arrays_out,dim=1)
norm_arrays_out = torch.cat(norm_arrays_out,dim=1)
age_arrays_out = torch.cat(age_arrays_out,dim=1)
smb_unperturbed_out = torch.cat(smb_unperturbed_out,dim=0)
smb_cnst_means_out = torch.cat(smb_cnst_means_out,dim=0)
smb_sds_out = torch.cat(smb_sds_out,dim=0)
smb_out = torch.cat(smb_out,dim=0)
bmb_out = torch.cat(bmb_out,dim=0)

with open(Path(loader._fol_path, loader._exp_path,"all_layers_test.p"), "wb") as fh:
    pickle.dump({"contour_arrays":contour_arrays_out,"norm_arrays":norm_arrays_out,"age_arrays":age_arrays_out}, fh)
with open(Path(loader._fol_path, loader._exp_path,"all_mbs_test.p"), "wb") as fh:
    pickle.dump({"smb_unperturbed_all":smb_unperturbed_out,"smb_cnst_means_all":smb_cnst_means_out,"smb_sds_all":smb_sds_out,"smb_all":smb_out,"bmb_all":bmb_out}, fh)